In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/AIM/clean_data.csv", lineterminator='\n')
df.head()

,id,txt,dialect
0,1175358310087892992,لكن بالنهاية ينتفض يغير,IQ
1,1175416117793349632,يعني هذا محسوب على البشر حيونه ووحشيه وتطلب...,IQ
2,1175450108898565888,مبين من كلامه خليجي,IQ
3,1175471073770573824,يسلملي مرورك وروحك الحلوه,IQ
4,1175496913145217024,وين هل الغيبه اخ محمد,IQ


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458000 entries, 0 to 457999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458000 non-null  int64 
 1   txt      458000 non-null  object
 2   dialect  458000 non-null  object
dtypes: int64(1), object(2)
memory usage: 10.5+ MB


In [ ]:
X = df['txt']
y = df['dialect']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

In [ ]:
vec = TfidfVectorizer(analyzer='word', max_df=.5)
clf = LinearSVC(class_weight = 'balanced')
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(max_df=0.5)),
                ('linearsvc', LinearSVC(class_weight='balanced'))])

In [ ]:
TfidfVectorizer(stop_words='english',
                              min_df=.01, max_df=.35, ngram_range=(1, 2))

TfidfVectorizer(max_df=0.35, min_df=0.01, ngram_range=(1, 2),
                stop_words='english')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
train_pred = pipe_tfidf.predict(x_train)
ac = accuracy_score(y_train, train_pred)
cm = confusion_matrix(y_train, train_pred)
report = classification_report(y_train, train_pred)

print("Train Accuracy is :",ac)
print(report)

Train Accuracy is : 0.9379066593886463
              precision    recall  f1-score   support

          AE       0.92      0.92      0.92     21029
          BH       0.93      0.91      0.92     20852
          DZ       0.92      0.96      0.94     12932
          EG       0.95      0.97      0.96     46049
          IQ       0.93      0.97      0.95     12441
          JO       0.93      0.89      0.91     22371
          KW       0.95      0.93      0.94     33746
          LB       0.94      0.96      0.95     22043
          LY       0.97      0.95      0.96     29168
          MA       0.95      0.98      0.96      9195
          OM       0.91      0.95      0.93     15253
          PL       0.95      0.88      0.91     34917
          QA       0.94      0.93      0.93     25038
          SA       0.93      0.94      0.93     21556
          SD       0.95      0.97      0.96     11466
          SY       0.91      0.95      0.93     12986
          TN       0.93      0.98      0.9

In [ ]:
pred = pipe_tfidf.predict(x_test)

In [ ]:
ac = accuracy_score(y_test, pred)
cm = confusion_matrix(y_test, pred)
report = classification_report(y_test, pred)

print("Test Accuracy is :",ac)
print(report)

Test Accuracy is : 0.5317139737991267
              precision    recall  f1-score   support

          AE       0.42      0.44      0.43      5267
          BH       0.37      0.34      0.36      5243
          DZ       0.54      0.59      0.56      3251
          EG       0.77      0.81      0.79     11587
          IQ       0.51      0.58      0.54      3056
          JO       0.42      0.37      0.39      5550
          KW       0.55      0.49      0.52      8363
          LB       0.61      0.66      0.64      5574
          LY       0.69      0.67      0.68      7331
          MA       0.60      0.62      0.61      2344
          OM       0.35      0.38      0.37      3863
          PL       0.56      0.47      0.51      8825
          QA       0.47      0.47      0.47      6031
          SA       0.40      0.41      0.40      5276
          SD       0.59      0.63      0.61      2968
          SY       0.38      0.39      0.39      3256
          TN       0.47      0.52      0.49

In [ ]:
import os
import json
from datetime import datetime
from threading import Thread, current_thread, get_ident
from typing import Dict, List, Union
import joblib

report = classification_report(y_test, pipe_tfidf.predict(x_test), output_dict=True)
classes = pipe_tfidf.classes_.tolist()
model_status = {}

def _update_status(status: str, classes: List[str] = [], evaluation: Dict = {}) -> None:
    model_status['status'] = status
    model_status['timestamp'] = datetime.now().isoformat()
    model_status['classes'] = classes
    model_status['evaluation'] = evaluation
    with open('model_status.json', 'w+') as file:
        json.dump(model_status, file, indent=2)


_update_status("Model Ready", classes, report)

In [ ]:
final_model = make_pipeline(vec, clf)
final_model.fit(X,y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(max_df=0.5)),
                ('linearsvc', LinearSVC(class_weight='balanced'))])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
train_pred = final_model.predict(X)
ac = accuracy_score(y, train_pred)
report = classification_report(y, train_pred)

print("Train Accuracy is :",ac)
print(report)

Train Accuracy is : 0.928264192139738
              precision    recall  f1-score   support

          AE       0.91      0.91      0.91     26296
          BH       0.92      0.90      0.91     26095
          DZ       0.91      0.95      0.93     16183
          EG       0.95      0.97      0.96     57636
          IQ       0.92      0.97      0.94     15497
          JO       0.92      0.88      0.90     27921
          KW       0.94      0.92      0.93     42109
          LB       0.94      0.95      0.94     27617
          LY       0.96      0.94      0.95     36499
          MA       0.94      0.97      0.96     11539
          OM       0.89      0.93      0.91     19116
          PL       0.94      0.87      0.90     43742
          QA       0.93      0.92      0.92     31069
          SA       0.91      0.93      0.92     26832
          SD       0.94      0.97      0.95     14434
          SY       0.90      0.94      0.92     16242
          TN       0.92      0.98      0.95

In [ ]:
import joblib

filename = 'ML_model.joblib'
joblib.dump(final_model, filename)

['ML_model.joblib']